In [ ]:
# Example 13: pseudo-range (2) (every 6s, using interpolation)
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget 

import json
import gpslib_tutorial as gpslib
import numpy as np
import matplotlib.pyplot as plt
import math

GPS_C  = 2.99792458e8
SMP_RATE = 2.048E6    
CODE_SAMPLES = 2048
N_CYC = 32
NGPS = N_CYC*CODE_SAMPLES
REC_POS = (4141657.6,604772.9,4796731.9)

# possible satellites 10,12,13,15,17,22,23,24,25
SAT0 = 24
SAT1 = 25
TOW_0 = 69447                # behind last code phase error

def loadCpLst(satNo):
    with open('../data/230914_gpsCP.json','r') as file:
        data=json.load(file)     
    return data[str(satNo)]

with open('../data/230914_gpsFrames.json','r') as file:
    FRAME_LST=json.load(file)  
    
satData,satPos,sfLst,towSet,cpLst,stLst,dist,smpTime = {},{},{},{},{},{},{},{}

for satNo in [SAT0,SAT1]:
    cpLst[satNo] = loadCpLst(satNo)
    satData[satNo] = gpslib.SatData(satNo)
    satPos[satNo] = gpslib.SatPos()
    sfLst[satNo] = list(filter(lambda item:item['SAT']==satNo and \
                               'ID' in item and item['tow']>=TOW_0,FRAME_LST))
    towSet[satNo] = set()
    stLst[satNo] = []
    ll = len(cpLst[satNo])-1
    k = 0
    for sf in sfLst[satNo]:
        satData[satNo].readSubframe(sf)         
        tow = sf['tow']
        st = sf['ST']                   # sample time of stream with preamble
        sno = st // NGPS                # stream number 
        while cpLst[satNo][k][0] < sno and k < ll:
            k += 1
        if cpLst[satNo][k][0] < sno:
            break
        elif cpLst[satNo][k][0] == sno:
            # get rid of integer code phase and use interpolated value
            st0 = (st // CODE_SAMPLES) * CODE_SAMPLES   
            stLst[satNo].append((tow,st0+cpLst[satNo][k][1]))   
            towSet[satNo].add(tow)        # set of valid values 
        
comTowSet = towSet[SAT0] & towSet[SAT1]   # common tows of both sats

for satNo in [SAT0,SAT1]:
    dist[satNo] = []
    smpTime[satNo] = []
    for tow,cpST in stLst[satNo]:
        if tow in comTowSet:
            x,y,z,d_st = satPos[satNo].ecefCoord(tow,satData[satNo].ephemData,
                                                 relCorr=False)
            d = np.sqrt((x-REC_POS[0])**2+(y-REC_POS[1])**2+(z-REC_POS[2])**2)
            dist[satNo].append(d)
            smpTime[satNo].append(cpST/SMP_RATE+d_st)
    dist[satNo] = np.asarray(dist[satNo])            
    smpTime[satNo] = np.asarray(smpTime[satNo])            
            
result = dist[SAT0]-dist[SAT1] - GPS_C*(smpTime[SAT0]-smpTime[SAT1])

mean = np.mean(result)
std  = np.std(result)

fig = plt.figure(figsize=(5,3))
fig.canvas.header_visible = False
plt.hist(result,bins=20,rwidth=0.8)
plt.ylabel('number / bin')
plt.xlabel('difference / m')
plt.tight_layout()
plt.show()
    